In [ ]:
import numpy as np 
import os 
import scipy.io
from sklearn.metrics import r2_score 
import matplotlib.pyplot as plt
import seaborn as sns
main_pathname = 'c:/Users/yaoyu/Documents/Epilepsy_research/'
window_length = 3
stride = 1
# Checking abnormal eigen/alpha values for a few patients, making sure FOS fits corrects 

patients = ['HUP64', 'HUP68','HUP72','HUP78','HUP86','MAYO010','MAYO011','MAYO016','MAYO020']
patients = ['HUP70']
for patient in patients: 
    for seizure in range(1, 9):
        for phase in ['interictal', 'ictal']:
            path = os.path.join(main_pathname, 'data_v2', patient, f'{phase}-block-{seizure}_parameters_3sec_1iter.mat')
            if(os.path.exists(path)):
                parameters = scipy.io.loadmat(path)
                raw_signal = scipy.io.loadmat(os.path.join(main_pathname, 'data', patient, f'{patient}-{phase}-block-{seizure}.mat'))

                alpha = parameters['alpha']
                A_matrix = parameters['A']
                xPred = parameters['xPred']
                evData = raw_signal['evData']
                sampling_rate = int(np.ceil(raw_signal['Fs'][0][0]))
                window_samples = sampling_rate*window_length
                stride_samples = sampling_rate*stride
                num_chns, data_points = evData.shape

                counts = np.zeros(data_points)
                evData_centered = np.zeros_like(evData)
                r2_windowed = np.zeros_like(evData)
                # need to locally center windowed raw data for r squared
                for window, start_idx in enumerate(range(0, data_points - window_samples + 1, stride_samples)):
                    X = evData[:, start_idx:start_idx + window_samples]
                    X = X - np.mean(X, axis = 1, keepdims=True)
                    evData_centered[:, start_idx:start_idx + window_samples] += X
                    counts[start_idx:start_idx + window_samples] += 1

                counts[counts == 0] = 1
                evData_centered /= counts[np.newaxis, :]

                # 1xchannel heatmap 

                r2_scores = [r2_score(evData_centered[ch], xPred[ch]) for ch in range(num_chns)]

                plt.figure(figsize=(12, 2))
                sns.heatmap([r2_scores], cmap='viridis', cbar=True, xticklabels=False)
                plt.title(f'R² per channel | {patient} Seizure {seizure} ({phase})')
                plt.xlabel('Channel Index')
                plt.yticks([])
                plt.show()

# temporal heatmap 

